## GAN
**생성**하는 모델, **적대적**으로 학습, **인공신경망** 모델

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [2]:
#hyper-parameters
EPOCHS = 100
BATCH_SIZE = 100
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
#fashion MNIST 데이터셋
trainset = datasets.FashionMNIST("./.data",
                                train = True,
                                download = True,
                                transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5,),(0.5,))
                                ]))
train_loader = torch.utils.data.DataLoader(
                dataset = trainset,
                batch_size = BATCH_SIZE,
                shuffle = True
)

# train_loader는 반복문에서 사용할 수 있으며, 반복 시 학습 이미지와 레이블을(이미지, 레이블) 튜플 형태로 반환

In [4]:
##생성자##

G = nn.Sequential(
    nn.Linear(64, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 784),
    nn.Tanh()) 

# 정규 분포에서 추출한 무작위 텐서(z)
# Tanh(): -1 에서 1 사이로 압축하는 역할

In [5]:
## 판별자

D = nn.Sequential(
    nn.Linear(784, 256),
    nn.LeakyReLU(),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256,1),
    nn.Sigmoid())

# sigmoid(): 0~1사이의 값을 가짐

In [6]:
# 모델의 가중치를 지정한 장치로 보내기
D = D.to(DEVICE)
G = G.to(DEVICE)

In [7]:
#Binary cross entropy 오차함수 & 생성자와 판별자를 최적화할 Adam 모듈
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr = 0.0002)
g_optimizer = optim.Adam(G.parameters(), lr = 0.0002)

In [8]:
# 판별자가 진짜를 진짜로 인식한 정확도 D(x)
# 가짜를 진짜로 인식한 정확도 D(G(z))

In [9]:
total_step = len(train_loader)
for epoch in range(EPOCHS):
    for i, (images,_) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(DEVICE)
        
        #'진짜', '가짜' 레이블 생성
        real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)
        
        #판별자가 진짜 이미지를 진짜로 인식하는 오차 계산
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        ##GENERATOR##
        #무작위 텐서로 가짜 이미지 생성
        z = torch.randn(BATCH_SIZE, 64).to(DEVICE)
        fake_images = G(z)

        #판별자가 가짜 이미지를 가짜로 인식하는 오차 계산
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        #진짜와 가짜 이미지를 갖고 낸 오차를 더해서 판별자의 오차 계산
        d_loss = d_loss_real + d_loss_fake
        
        #역전파 알고리즘으로 판별자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        #생성자가 판별자를 속였는지에 대한 오차 계산
        fake_images = G(z)
        outputs = D(fake_images)
        g_loss = criterion(outputs, real_labels)
        
        #역전파 알고리즘으로 생성자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    #학습 진행 알아보기
    print("이폭 [{}/{}] d_loss:{:.4f} g_loss:{:.4f} D(x):{:.2f} D(G(z)):{:.2f}".format(epoch, EPOCHS, 
                                                                        d_loss.item(), g_loss.item(), 
                                                                        real_score.mean().item(), fake_score.mean().item()))

이폭 [0/100] d_loss:0.0514 g_loss:4.7764 D(x):0.97 D(G(z)):0.01
이폭 [1/100] d_loss:0.0187 g_loss:6.4155 D(x):0.99 D(G(z)):0.01
이폭 [2/100] d_loss:0.0233 g_loss:5.2801 D(x):0.99 D(G(z)):0.01
이폭 [3/100] d_loss:0.0785 g_loss:6.0816 D(x):0.98 D(G(z)):0.04
이폭 [4/100] d_loss:0.0329 g_loss:5.8901 D(x):0.98 D(G(z)):0.01
이폭 [5/100] d_loss:0.1031 g_loss:6.4877 D(x):0.97 D(G(z)):0.02
이폭 [6/100] d_loss:0.0774 g_loss:5.7627 D(x):0.98 D(G(z)):0.04
이폭 [7/100] d_loss:0.1270 g_loss:3.9920 D(x):0.98 D(G(z)):0.08
이폭 [8/100] d_loss:0.5431 g_loss:3.1984 D(x):0.89 D(G(z)):0.09
이폭 [9/100] d_loss:0.0873 g_loss:5.0862 D(x):0.96 D(G(z)):0.02
이폭 [10/100] d_loss:0.2256 g_loss:3.8360 D(x):0.94 D(G(z)):0.09
이폭 [11/100] d_loss:0.3995 g_loss:2.9484 D(x):0.91 D(G(z)):0.07
이폭 [12/100] d_loss:0.5173 g_loss:2.8338 D(x):0.87 D(G(z)):0.08
이폭 [13/100] d_loss:0.4314 g_loss:3.1170 D(x):0.92 D(G(z)):0.15
이폭 [14/100] d_loss:0.3808 g_loss:2.1649 D(x):0.92 D(G(z)):0.17
이폭 [15/100] d_loss:0.2119 g_loss:3.2866 D(x):0.94 D(G(z)):0.10
이폭

50%로 가까워질수록 둘다 잘 학습된 것임

In [10]:
import numpy as np
z = torch.randn(BATCH_SIZE, 64).to(DEVICE)
fake_images = G(z)
for i in range(10):
    print(type(fake_images_img))
    print(type(fake_images_img.data))
    fake_images_img = np.reshape(fake_images.data.cpu().numpy()[i],(28,28))
    plt.imshow(fake_images_img, cmap = 'gray')
    plt.show()

NameError: name 'fake_images_img' is not defined

# cGAN
위의 모델은 이미지를 무작위로 생성하는 것임  
출력할 아이템의 종류를 사용자로 부터 입력을 받아 그에 해당하는 이미지를 생성하는 모델 '**조건부 GAN**', '**cGAN**' (생성제어)  
cGAN은 생성자와 판별자에 **레이블의 정보**가 같이 들어간다

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# 하이퍼파라미터
EPOCHS = 100
BATCH_SIZE = 100
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
trainset = datasets.FashionMNIST(
    './.data',
    train=True,
    download=True,
    transform=transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.5,), (0.5,))]))

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True
)

- 생성자에 입력할때 100 + 10(class의 종류)
- embed() 함수는 배치 x1 크기의 레이블 텐서를 받아 배치 x10 의 연속적인 텐서로 변환
    - 똑같은 레이블은 항상 똑같은 텐서를 만들기 때문에 단순히 연속적인 텐서를 레이블 값에 단순 대입(mapping)한다고 생각하면 됨

In [ ]:
# 생성자 (Generator)
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embed = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(110, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        c = self.embed(labels)
        x = torch.cat([z, c], 1)
        return self.model(x)

In [ ]:
# 판별자 (Discriminator)
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embed = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(794, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, labels):
        c = self.embed(labels)
        x = torch.cat([x, c], 1)
        return self.model(x)

In [ ]:
D = Discriminator().to(DEVICE)
G = Generator().to(DEVICE)

In [ ]:
#Binary cross entropy 오차함수 & 생성자와 판별자를 최적화할 Adam 모듈
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr =0.0002)
g_optimizer = optim.Adam(G.parameters(), lr =0.0002)

In [ ]:
total_step = len(train_loader)
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(DEVICE)
        
        # '진짜'와 '가짜' 레이블 생성
        real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)

        # 판별자가 진짜 이미지를 진짜로 인식하는 오차 계산 (데이터셋 레이블 입력)
        labels = labels.to(DEVICE)
        outputs = D(images, labels)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
    
        # 무작위 텐서와 무작위 레이블을 생성자에 입력해 가짜 이미지 생성
        z = torch.randn(BATCH_SIZE, 100).to(DEVICE)
        g_label = torch.randint(0, 10, (BATCH_SIZE,)).to(DEVICE)
        fake_images = G(z, g_label)
        
        # 판별자가 가짜 이미지를 가짜로 인식하는 오차 계산
        outputs = D(fake_images, g_label)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # 진짜와 가짜 이미지를 갖고 낸 오차를 더해서 판별자의 오차 계산
        d_loss = d_loss_real + d_loss_fake
        
        # 역전파 알고리즘으로 판별자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 생성자가 판별자를 속였는지에 대한 오차 계산(무작위 레이블 입력)
        fake_images = G(z, g_label)
        outputs = D(fake_images, g_label)
        g_loss = criterion(outputs, real_labels)

        # 역전파 알고리즘으로 생성자 모델의 학습을 진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    print('이폭 [{}/{}] d_loss:{:.4f} g_loss: {:.4f} D(x):{:.2f} D(G(z)):{:.2f}'
          .format(epoch,
                  EPOCHS,
                  d_loss.item(),
                  g_loss.item(),
                  real_score.mean().item(),
                  fake_score.mean().item()))

In [ ]:
# 만들고 싶은 아이템 생성하고 시각화하기
item_number = 9 # 아이템 번호
z = torch.randn(1, 100).to(DEVICE) # 배치 크기 1
g_label = torch.full((1,), item_number, dtype=torch.long).to(DEVICE)
sample_images = G(z, g_label)

sample_images_img = np.reshape(sample_images.data.cpu().numpy()
                               [0],(28, 28))
plt.imshow(sample_images_img, cmap = 'gray')
plt.show()